# First look into the available Dataset

#### Dataset: "Global Shark Attacks"
*Sorce: Kaggle[https://www.kaggle.com/teajay/global-shark-attacks]*


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
pd.read_csv("./assets/attacks.csv", nrows=2)

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
0,2018.06.25,25-Jun-2018,2018,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,...,White shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.25,2018.06.25,6303,NaN,NaN
1,2018.06.18,18-Jun-2018,2018,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson�McNeely,F,...,NaN,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.18,2018.06.18,6302,NaN,NaN


In [3]:
shark_df = pd.read_csv('./assets/attacks.csv', encoding='latin1')
pd.set_option("max_colwidth", 60)

In [4]:
shark_df.shape

(25723, 24)

In [5]:
shark_df.head()

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
0,2018.06.25,25-Jun-2018,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,...,White shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_directory/20...,http://sharkattackfile.net/spreadsheets/pdf_directory/20...,2018.06.25,2018.06.25,6303.0,NaN,NaN
1,2018.06.18,18-Jun-2018,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,...,NaN,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_directory/20...,http://sharkattackfile.net/spreadsheets/pdf_directory/20...,2018.06.18,2018.06.18,6302.0,NaN,NaN
2,2018.06.09,09-Jun-2018,2018.0,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,...,NaN,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_directory/20...,http://sharkattackfile.net/spreadsheets/pdf_directory/20...,2018.06.09,2018.06.09,6301.0,NaN,NaN
3,2018.06.08,08-Jun-2018,2018.0,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,...,2 m shark,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_directory/20...,http://sharkattackfile.net/spreadsheets/pdf_directory/20...,2018.06.08,2018.06.08,6300.0,NaN,NaN
4,2018.06.04,04-Jun-2018,2018.0,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,...,"Tiger shark, 3m",A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_directory/20...,http://sharkattackfile.net/spreadsheets/pdf_directory/20...,2018.06.04,2018.06.04,6299.0,NaN,NaN


In [6]:
shark_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25723 entries, 0 to 25722
Data columns (total 24 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Case Number             8702 non-null   object 
 1   Date                    6302 non-null   object 
 2   Year                    6300 non-null   float64
 3   Type                    6298 non-null   object 
 4   Country                 6252 non-null   object 
 5   Area                    5847 non-null   object 
 6   Location                5762 non-null   object 
 7   Activity                5758 non-null   object 
 8   Name                    6092 non-null   object 
 9   Sex                     5737 non-null   object 
 10  Age                     3471 non-null   object 
 11  Injury                  6274 non-null   object 
 12  Fatal (Y/N)             5763 non-null   object 
 13  Time                    2948 non-null   object 
 14  Species                 3464 non-null 

In [7]:
shark_df.isnull().sum().sort_values(ascending=False)

Unnamed: 22               25722
Unnamed: 23               25721
Time                      22775
Species                   22259
Age                       22252
Sex                       19986
Activity                  19965
Location                  19961
Fatal (Y/N)               19960
Area                      19876
Name                      19631
Country                   19471
Injury                    19449
Investigator or Source    19438
Type                      19425
Year                      19423
href formula              19422
Date                      19421
pdf                       19421
href                      19421
Case Number.1             19421
Case Number.2             19421
original order            19414
Case Number               17021
dtype: int64

In [8]:
# source csv file (attacks.csv found @ Kaggle.com) contains almost 3/4 of empty rows

shark_df[shark_df.duplicated(keep=False)].head()

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
6309,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6310,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6311,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6312,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6313,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
shark_df.drop_duplicates(inplace=True)

In [10]:
shark_df.shape

(6312, 24)

In [11]:
shark_df.columns

Index(['Case Number', 'Date', 'Year', 'Type', 'Country', 'Area', 'Location',
       'Activity', 'Name', 'Sex ', 'Age', 'Injury', 'Fatal (Y/N)', 'Time',
       'Species ', 'Investigator or Source', 'pdf', 'href formula', 'href',
       'Case Number.1', 'Case Number.2', 'original order', 'Unnamed: 22',
       'Unnamed: 23'],
      dtype='object')

In [12]:
shark_df.columns = shark_df.columns.str.lower()

In [13]:
shark_df.columns

Index(['case number', 'date', 'year', 'type', 'country', 'area', 'location',
       'activity', 'name', 'sex ', 'age', 'injury', 'fatal (y/n)', 'time',
       'species ', 'investigator or source', 'pdf', 'href formula', 'href',
       'case number.1', 'case number.2', 'original order', 'unnamed: 22',
       'unnamed: 23'],
      dtype='object')

In [14]:
# keeping usefull columns only

shark_df = shark_df[
    ['case number', 'type', 'country', 'area', 'location',
     'activity', 'sex ', 'age', 'fatal (y/n)', 'species ',
     'href formula']
]

shark_df.head()

,case number,type,country,area,location,activity,sex,age,fatal (y/n),species,href formula
0,2018.06.25,Boating,USA,California,"Oceanside, San Diego County",Paddling,F,57,N,White shark,http://sharkattackfile.net/spreadsheets/pdf_directory/20...
1,2018.06.18,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,F,11,N,NaN,http://sharkattackfile.net/spreadsheets/pdf_directory/20...
2,2018.06.09,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,M,48,N,NaN,http://sharkattackfile.net/spreadsheets/pdf_directory/20...
3,2018.06.08,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,M,NaN,N,2 m shark,http://sharkattackfile.net/spreadsheets/pdf_directory/20...
4,2018.06.04,Provoked,MEXICO,Colima,La Ticla,Free diving,M,NaN,N,"Tiger shark, 3m",http://sharkattackfile.net/spreadsheets/pdf_directory/20...


In [15]:
shark_df['case number'] = shark_df['case number'].astype(str)

In [16]:
shark_df['year'] = shark_df['case number'].str[0:4]
shark_df['month'] = shark_df['case number'].str[5:7]
shark_df.drop(['case number'], axis=1, inplace=True)

In [17]:
shark_df.head()

,type,country,area,location,activity,sex,age,fatal (y/n),species,href formula,year,month
0,Boating,USA,California,"Oceanside, San Diego County",Paddling,F,57,N,White shark,http://sharkattackfile.net/spreadsheets/pdf_directory/20...,2018,06
1,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,F,11,N,NaN,http://sharkattackfile.net/spreadsheets/pdf_directory/20...,2018,06
2,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,M,48,N,NaN,http://sharkattackfile.net/spreadsheets/pdf_directory/20...,2018,06
3,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,M,NaN,N,2 m shark,http://sharkattackfile.net/spreadsheets/pdf_directory/20...,2018,06
4,Provoked,MEXICO,Colima,La Ticla,Free diving,M,NaN,N,"Tiger shark, 3m",http://sharkattackfile.net/spreadsheets/pdf_directory/20...,2018,06


In [18]:
# rename important columns

shark_df = shark_df.rename(columns={
    'sex ':'sex',
    'fatal (y/n)':'fatal',
    'species ':'species',
    'href formula':'link'
})

shark_df.head(2)

,type,country,area,location,activity,sex,age,fatal,species,link,year,month
0,Boating,USA,California,"Oceanside, San Diego County",Paddling,F,57,N,White shark,http://sharkattackfile.net/spreadsheets/pdf_directory/20...,2018,06
1,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,F,11,N,NaN,http://sharkattackfile.net/spreadsheets/pdf_directory/20...,2018,06


In [19]:
# reorder columns

shark_df = shark_df[[
    'year', 'month', 'type', 'country',
    'area', 'location', 'activity', 'sex',
    'age', 'fatal', 'species', 'link'
]]

shark_df.head(2)

,year,month,type,country,area,location,activity,sex,age,fatal,species,link
0,2018,06,Boating,USA,California,"Oceanside, San Diego County",Paddling,F,57,N,White shark,http://sharkattackfile.net/spreadsheets/pdf_directory/20...
1,2018,06,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,F,11,N,NaN,http://sharkattackfile.net/spreadsheets/pdf_directory/20...


In [20]:
shark_df.isnull().sum().sort_values(ascending=False)

species     2848
age         2841
sex          575
activity     554
location     550
fatal        549
area         465
country       60
type          14
link          11
year           0
month          0
dtype: int64